In [1]:
import tensorflow as tf
import numpy as np
import os

import tensorflow.contrib.eager as tfe

tf.enable_eager_execution()

/usr/lib/python3.4/importlib/_bootstrap.py:321: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  return f(*args, **kwds)


Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
#資料庫

train_dataset_url = "http://download.tensorflow.org/data/iris_training.csv"

train_dataset_fp = tf.keras.utils.get_file(fname=os.path.basename(train_dataset_url),
                                           origin=train_dataset_url)

print("Local copy of the dataset file: {}".format(train_dataset_fp))

Local copy of the dataset file: /tmp/.keras/datasets/iris_training.csv


In [3]:
os.path.basename(train_dataset_url)

'iris_training.csv'

In [4]:
train_dataset_url

'http://download.tensorflow.org/data/iris_training.csv'

In [5]:
train_dataset_fp

'/tmp/.keras/datasets/iris_training.csv'

In [6]:
# 驚嘆號在開頭 python 調用系統的指令

!head -n10 {train_dataset_fp}

120,4,setosa,versicolor,virginica
6.4,2.8,5.6,2.2,2
5.0,2.3,3.3,1.0,1
4.9,2.5,4.5,1.7,2
4.9,3.1,1.5,0.1,0
5.7,3.8,1.7,0.3,0
4.4,3.2,1.3,0.2,0
5.4,3.4,1.5,0.4,0
6.9,3.1,5.1,2.3,2
6.7,3.1,4.4,1.4,1


In [7]:
# 調系統統

!who

lewis    tty1         2018-04-05 15:27
lewis    pts/1        2018-04-05 15:28 (192.168.0.101)


In [8]:
def parse_csv(line):
    example_defaults = [[0.], [0.], [0.], [0.], [0]]
    parsed_line = tf.decode_csv(line, example_defaults)
    features = tf.reshape(parsed_line[:-1], shape=(4,))
    label = tf.reshape(parsed_line[-1], shape=())
    
    return features, label

In [9]:
train_dataset = tf.data.TextLineDataset(train_dataset_fp)
train_dataset = train_dataset.skip(1)             # skip the first header row
train_dataset = train_dataset.map(parse_csv)      # parse each row
train_dataset = train_dataset.shuffle(buffer_size=1000)  # randomize
train_dataset = train_dataset.batch(32)

In [10]:
train_dataset

<BatchDataset shapes: ((?, 4), (?,)), types: (tf.float32, tf.int32)>

In [11]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation="relu", input_shape=(4,)),  # input shape required
    tf.keras.layers.Dense(10, activation="relu"),
    tf.keras.layers.Dense(3)
])

In [12]:
def loss(model, x, y):
    y_ = model(x)
    
    return tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)

In [13]:
def grad(model, inputs, targets):
    with tfe.GradientTape() as tape:
        loss_value = loss(model, inputs, targets)
    
    return tape.gradient(loss_value, model.variables)

In [14]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)

In [15]:
train_loss_results = []
train_accuracy_results = []

In [16]:
num_epochs = 201

In [17]:
for epoch in range(num_epochs):
    epoch_loss_avg = tfe.metrics.Mean()
    epoch_accuracy = tfe.metrics.Accuracy()
    
    for x, y in tfe.Iterator(train_dataset):
        grads = grad(model, x, y)
        optimizer.apply_gradients(zip(grads, model.variables), global_step=tf.train.get_or_create_global_step())
        
        epoch_loss_avg(loss(model, x, y))  # add current batch loss
        epoch_accuracy(tf.argmax(model(x), axis=1, output_type=tf.int32), y)
        
    train_loss_results.append(epoch_loss_avg.result())
    train_accuracy_results.append(epoch_accuracy.result())
    
    if epoch % 50 == 0:
        print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(epoch, epoch_loss_avg.result(), epoch_accuracy.result()))

Epoch 000: Loss: 1.337, Accuracy: 30.000%
Epoch 050: Loss: 0.514, Accuracy: 80.833%
Epoch 100: Loss: 0.340, Accuracy: 93.333%
Epoch 150: Loss: 0.249, Accuracy: 95.833%
Epoch 200: Loss: 0.176, Accuracy: 96.667%


In [18]:
train_loss_results

[<tf.Tensor: id=781, shape=(), dtype=float64, numpy=1.3367856442928314>,
 <tf.Tensor: id=1453, shape=(), dtype=float64, numpy=1.2112858593463898>,
 <tf.Tensor: id=2117, shape=(), dtype=float64, numpy=1.1024219989776611>,
 <tf.Tensor: id=2781, shape=(), dtype=float64, numpy=1.0331390351057053>,
 <tf.Tensor: id=3445, shape=(), dtype=float64, numpy=0.999743789434433>,
 <tf.Tensor: id=4109, shape=(), dtype=float64, numpy=0.9730575233697891>,
 <tf.Tensor: id=4773, shape=(), dtype=float64, numpy=0.9604362696409225>,
 <tf.Tensor: id=5437, shape=(), dtype=float64, numpy=0.943862795829773>,
 <tf.Tensor: id=6101, shape=(), dtype=float64, numpy=0.9260620027780533>,
 <tf.Tensor: id=6765, shape=(), dtype=float64, numpy=0.9165887236595154>,
 <tf.Tensor: id=7429, shape=(), dtype=float64, numpy=0.9039689153432846>,
 <tf.Tensor: id=8093, shape=(), dtype=float64, numpy=0.8904195874929428>,
 <tf.Tensor: id=8757, shape=(), dtype=float64, numpy=0.8798197656869888>,
 <tf.Tensor: id=9421, shape=(), dtype=flo